## Setup

Please make sure you have outputs from 'setup_for_notebooks.ipynb' on your current directory.

In [1]:
import torch
from torch.utils.data import DataLoader

model = torch.load("resnet50.pt").eval()
dataset = torch.load("imagenet_sample.pt")
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
x_batch, y_batch = next(iter(dataloader))
p_batch = model(x_batch).argmax(1)

/home/gkim/Projects/pnpxai-demo/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Now we have 16 samples to be explained.

In [2]:
print("x_batch.shape", x_batch.shape)
print("y_batch.shape", y_batch.shape)
print("p_batch.shape", p_batch.shape)

x_batch.shape torch.Size([16, 3, 224, 224])
y_batch.shape torch.Size([16])
p_batch.shape torch.Size([16])


## Explain

In this demo, I've tried to design `ExplainerInterface` to be initialized with untunable parameters and run explanation with tunable parameters. I guess this should be replaced to a wrapper one without constraints on input params because this design unecessarily restricts user's customizability. By the way, let's see how the explainers in this demo work.

In [3]:
from pnpxai.core.explainers.lrp import LRPEpsilonPlus
from pnpxai.core.explainers.gradcam import GradCAMpp

In [4]:
# lrp epsilon plus
explainer = LRPEpsilonPlus(model=model)

a_batch_lrpep_default = explainer.attribute(x_batch, p_batch) # default
a_batch_lrpep = explainer.attribute(x_batch, p_batch, epsilon=.5) # set tunable parameters

In [5]:
# gradcampp
explainer = GradCAMpp(model=model)
a_batch_gcampp_default = explainer.attribute(inputs=x_batch, targets=p_batch)

/home/gkim/Projects/pnpxai-demo/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


I set the target layer as an untunable parameter for explainer.

In [6]:
explainer = GradCAMpp(model=model, target_layer="layer4")
a_batch_gcampp_tl = explainer.attribute(inputs=x_batch, targets=p_batch)